In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
import keras_tuner
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import  HyperParameters

In [3]:
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

In [4]:
train_generator = train_datagen.flow_from_directory(
    'C:\\Users\\weiyi\\Desktop\\dataset\\horse-or-human',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    'C:\\Users\\weiyi\\Desktop\\dataset\\validation-horse-or-human',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'C:\\Users\\weiyi\\Desktop\\dataset\\horse-or-human'

In [6]:
hp=HyperParameters()

In [7]:
def build_model(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(hp.Choice('num_filters_layer0',values=[16,64],
        default=16),(3,3), activation='relu',input_shape=(150,150,3)))
    model.add(tf.keras.layers.MaxPool2D(2,2))
    for i in range(hp.Int("num_conv_layers",1,3)):
        model.add(tf.keras.layers.Conv2D(hp.Choice(f'num_filters_layer{i}',values=[16,64],default=16),(3,3), activation='relu'))
        model.add(tf.keras.layers.MaxPool2D(2,2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(hp.Int("hidden_units",128,512,step=32), activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer=RMSprop(learning_rate=0.001),metrics=['acc'])
    return model

In [8]:
tuner=Hyperband(
    build_model,
    objective='val_acc',
    max_epochs=15,
    directory='horse_human_params',
    hyperparameters=hp,
    project_name='my_horse_human_project'
)

In [58]:
tuner.search(
    train_generator,
    epochs=10,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=8)

Trial 30 Complete [00h 01m 05s]
val_acc: 0.87890625

Best val_acc So Far: 0.921875
Total elapsed time: 00h 11m 52s
INFO:tensorflow:Oracle triggered exit


In [61]:
best_hps=tuner.get_best_hyperparameters(1)[0]
print(best_hps.values)
model = tuner.hypermodel.build(best_hps)
model.summary()

{'num_filters_layer0': 16, 'num_conv_layers': 3, 'hidden_units': 192, 'num_filters_layer1': 64, 'tuner/epochs': 5, 'tuner/initial_epoch': 2, 'tuner/bracket': 2, 'tuner/round': 1, 'num_filters_layer2': 16, 'tuner/trial_id': '0001'}
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 72, 72, 16)        2320      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 34, 34, 64)        9280      
_____________________